In [900]:
import numpy as np
import pandas as pd
import seaborn as sns
from datasist.structdata import detect_outliers
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
import joblib

In [880]:
df = pd.read_csv('..\\..\\datasets\\titanic_full.csv', na_values=['missing', 'xx', '?', 'x', '-'])
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [881]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [882]:
df.drop(['Name', 'Ticket', 'Cabin', 'PassengerId'], axis=1, inplace=True)
df

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S
...,...,...,...,...,...,...,...,...
886,0,2,male,27.0,0,0,13.0000,S
887,1,1,female,19.0,0,0,30.0000,S
888,0,3,female,NaN,1,2,23.4500,S
889,1,1,male,26.0,0,0,30.0000,C


In [883]:
df.describe()

,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [884]:
numerical_cols = ['Age', 'Fare']
categorical_cols = df.drop(numerical_cols, axis=1)

In [885]:
for col in categorical_cols:
    print(f'{col} =====> {df[col].unique()}')

Survived =====> [0 1]
Pclass =====> [3 1 2]
Sex =====> ['male' 'female']
SibSp =====> [1 0 3 4 2 5 8]
Parch =====> [0 1 2 5 3 4 6]
Embarked =====> ['S' 'C' 'Q' nan]


In [886]:
for col in numerical_cols:
    out_indices = detect_outliers(df, 0, [col])
    df.loc[out_indices, col] = np.nan

In [887]:
df.isna().sum()

Survived      0
Pclass        0
Sex           0
Age         177
SibSp         0
Parch         0
Fare        116
Embarked      2
dtype: int64

In [888]:
imputer = KNNImputer()
df['Age'] = imputer.fit_transform(df[['Age']])
df['Fare'] = imputer.fit_transform(df[['Fare']])
imputer = SimpleImputer(strategy='most_frequent')
df['Embarked'] = imputer.fit_transform(df[['Embarked']])
df.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64

In [889]:
df = pd.get_dummies(df, columns=['Embarked', 'Sex'], drop_first=True)
df

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_male
0,0,3,22.000000,1,0,7.250000,0,1,1
1,1,1,38.000000,1,0,17.822091,0,0,0
2,1,3,26.000000,0,0,7.925000,0,1,0
3,1,1,35.000000,1,0,53.100000,0,1,0
4,0,3,35.000000,0,0,8.050000,0,1,1
...,...,...,...,...,...,...,...,...,...
886,0,2,27.000000,0,0,13.000000,0,1,1
887,1,1,19.000000,0,0,30.000000,0,1,0
888,0,3,29.699118,1,2,23.450000,0,1,0
889,1,1,26.000000,0,0,30.000000,0,0,1


In [890]:
df['Age'] = df['Age'].apply(round)
df

,Survived,Pclass,Age,SibSp,Parch,Fare,Embarked_Q,Embarked_S,Sex_male
0,0,3,22,1,0,7.250000,0,1,1
1,1,1,38,1,0,17.822091,0,0,0
2,1,3,26,0,0,7.925000,0,1,0
3,1,1,35,1,0,53.100000,0,1,0
4,0,3,35,0,0,8.050000,0,1,1
...,...,...,...,...,...,...,...,...,...
886,0,2,27,0,0,13.000000,0,1,1
887,1,1,19,0,0,30.000000,0,1,0
888,0,3,30,1,2,23.450000,0,1,0
889,1,1,26,0,0,30.000000,0,0,1


In [891]:
x = df.drop('Survived', axis=1)
y = df['Survived']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
print(f"{x.shape} || {y.shape}")
print(f"{x_train.shape} || {x_test.shape} || {y_train.shape} || {y_test.shape}")

(891, 8) || (891,)
(801, 8) || (90, 8) || (801,) || (90,)


In [892]:
y_train.value_counts()

0    486
1    315
Name: Survived, dtype: int64

In [893]:
sampler = SMOTE()
x_train, y_train = sampler.fit_resample(x_train, y_train)

In [894]:
y_train.value_counts()

0    486
1    486
Name: Survived, dtype: int64

In [895]:
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)

In [896]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, fbeta_score, classification_report

In [897]:
models = {
    "LR": LogisticRegression(),
    "KNN": KNeighborsClassifier(),
    "SVC": SVC(),
    "DT": DecisionTreeClassifier(),
    "RF": RandomForestClassifier(),
    "XGB": XGBClassifier(),
    "Naive Bayes": GaussianNB()
}

In [898]:
for name, model in models.items():
    print(f'Training Model {name} \n--------------')
    model.fit(x_train, y_train)
    y_pred = model.predict(x_test)
    print(f'Training Accuracy: {accuracy_score(y_train, model.predict(x_train))}')
    print(f'Testing Accuracy: {accuracy_score(y_test, y_pred)}')
    print(f'Testing Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')
    print(f'Testing Recall: {recall_score(y_test, y_pred)}')    
    print(f'Testing Precesion: {precision_score(y_test, y_pred)}')
    print(f'Testing F-1: {f1_score(y_test, y_pred)}')
    print(f'Testing F-Beta: {fbeta_score(y_test, y_pred, beta=2)}')
    print('-'*30)

Training Model LR 
--------------
Training Accuracy: 0.8055555555555556
Testing Accuracy: 0.8555555555555555
Testing Confusion Matrix: 
[[53 10]
 [ 3 24]]
Testing Recall: 0.8888888888888888
Testing Precesion: 0.7058823529411765
Testing F-1: 0.7868852459016393
Testing F-Beta: 0.8450704225352113
------------------------------
Training Model KNN 
--------------
Training Accuracy: 0.8600823045267489
Testing Accuracy: 0.8777777777777778
Testing Confusion Matrix: 
[[56  7]
 [ 4 23]]
Testing Recall: 0.8518518518518519
Testing Precesion: 0.7666666666666667
Testing F-1: 0.8070175438596491
Testing F-Beta: 0.8333333333333334
------------------------------
Training Model SVC 
--------------
Training Accuracy: 0.8436213991769548
Testing Accuracy: 0.9
Testing Confusion Matrix: 
[[58  5]
 [ 4 23]]
Testing Recall: 0.8518518518518519
Testing Precesion: 0.8214285714285714
Testing F-1: 0.8363636363636364
Testing F-Beta: 0.8455882352941175
------------------------------
Training Model DT 
--------------
T

C:\Users\moham\anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Testing F-Beta: 0.8214285714285714
------------------------------
Training Model Naive Bayes 
--------------
Training Accuracy: 0.7973251028806584
Testing Accuracy: 0.8
Testing Confusion Matrix: 
[[47 16]
 [ 2 25]]
Testing Recall: 0.9259259259259259
Testing Precesion: 0.6097560975609756
Testing F-1: 0.7352941176470589
Testing F-Beta: 0.8389261744966443
------------------------------


In [899]:
model = SVC()
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
print(f'Training Accuracy: {accuracy_score(y_train, model.predict(x_train))}')
print(f'Testing Accuracy: {accuracy_score(y_test, y_pred)}')
print(f'Testing Recall: {recall_score(y_test, y_pred)}')    
print(f'Testing Precesion: {precision_score(y_test, y_pred)}')

Training Accuracy: 0.8436213991769548
Testing Accuracy: 0.9
Testing Recall: 0.8518518518518519
Testing Precesion: 0.8214285714285714


In [901]:
joblib.dump(model, 'model.h5')
joblib.dump(scaler, 'scaler.h5')

['scaler.h5']